In [1]:
%%capture --no-stderr
%pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python nomic[local] langchain-text-splitters

In [2]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "ture"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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>"

In [3]:
local_llm = "llama3"

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [18]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    "https://www.impactmybiz.com/blog/what-are-nist-security-standards/"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=50, chunk_overlap=0
)

doc_splits = text_splitter.split_documents(docs_list)

In [31]:
# ! pip install PyPDF2

In [36]:
import PyPDF2
def extract_text_from_pdf(pdf_path):
    reader = PyPDF2.PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

pdf_text = extract_text_from_pdf('/mnt/e/Sikh Faith Book.pdf')
print(pdf_text)

Sikh Faith - Sikh Faith - Sikh Faith - Sikh Faith - Sikh Faith - An EpitomeAn EpitomeAn EpitomeAn EpitomeAn Epitome
of Inter-Faithof Inter-Faithof Inter-Faithof Inter-Faithof Inter-Faith
for Divine Realisationfor Divine Realisationfor Divine Realisationfor Divine Realisationfor Divine Realisation< siqgurpRswid ]< siqgurpRswid ]< siqgurpRswid ]< siqgurpRswid ]< siqgurpRswid ]
Sikh Faith - Sikh Faith - Sikh Faith - Sikh Faith - Sikh Faith - An EpitomeAn EpitomeAn EpitomeAn EpitomeAn Epitome
of Inter-Faithof Inter-Faithof Inter-Faithof Inter-Faithof Inter-Faith
for Divine Realisationfor Divine Realisationfor Divine Realisationfor Divine Realisationfor Divine Realisation
srb Drm mih sRyst Drmu ]
hir ko nwmu jip inrml  krmu ]
Sarab dharam meh(i) sr-aisatdharam(u)
Har(i) ko Naam(u) jap(i) nirmal karam(u). (266)
(Among all the religions, the supreme religion, in reality , is to recite the Divine
Name with full love and devotion; imbibe spiritual love with full faith: render selfless
service t

In [37]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
doc = Document(page_content=pdf_text)

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=50, chunk_overlap=0
)

# Split the document
doc_splits = text_splitter.split_documents([doc])

# Print the chunks
for i, split_doc in enumerate(doc_splits):
    print(f"Chunk {i+1}: {split_doc.page_content[:100]}...")


Chunk 1: Sikh Faith - Sikh Faith - Sikh Faith - Sikh Faith - Sikh Faith - An EpitomeAn EpitomeAn EpitomeAn Ep...
Chunk 2: of Inter-Faithof Inter-Faithof Inter-Faithof Inter-Faithof Inter-Faith...
Chunk 3: for Divine Realisationfor Divine Realisationfor Divine Realisationfor Divine Realisationfor Divine R...
Chunk 4: siqgurpRswid ]< siqgurpRswid ]...
Chunk 5: Sikh Faith - Sikh Faith - Sikh Faith - Sikh Faith - Sikh Faith - An EpitomeAn EpitomeAn EpitomeAn Ep...
Chunk 6: of Inter-Faithof Inter-Faithof Inter-Faithof Inter-Faithof Inter-Faith
for Divine Realisationfor Div...
Chunk 7: srb Drm mih sRyst Drmu ]
hir ko nwmu jip inrml  krmu ]...
Chunk 8: Sarab dharam meh(i) sr-aisatdharam(u)
Har(i) ko Naam(u) jap(i) nirmal karam(u). (266)...
Chunk 9: (Among all the religions, the supreme religion, in reality , is to recite the Divine
Name with full ...
Chunk 10: service to humanity; so as to eradicate ego completely and thus merge with Him)...
Chunk 11: Iqbal Singh (Baba)Iqbal Singh (Baba)Iqbal

In [32]:
# pdf_text = (
#     "This is a larger sample text. " * 50 +  # Repeat to make the text longer
#     "This is another segment of the text. " * 50
# )

# Create a Document object
doc = Document(page_content=pdf_text)

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=50, chunk_overlap=0
)

Split the document
doc_splits = text_splitter.split_documents([doc])

# Print the chunks
for i, split_doc in enumerate(doc_splits):
    print(f"Chunk {i+1}: {split_doc.page_content[:100]}...")  # Print the first 100 characters of each chunk


SyntaxError: invalid syntax (1643990708.py, line 14)

In [38]:
# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
)
retriever = vectorstore.as_retriever()

In [43]:
# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
    persist_directory="./chroma_data",
)
retriever = vectorstore.as_retriever()

In [15]:
# from langchain import PromptTemplate, LLMChain
# from langchain.llms import OpenAI
# from langchain.output_parsers import JsonOutputParser

# # Assuming llm is already defined, e.g., llm = OpenAI()

# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
#     You are an assistant answering questions based on the provided context. 
#     Use the following CONTEXT to answer the QUESTION at the end.
#     If you don't know the answer, just say "I don't know" without making up any information.
#     Provide your answer directly without any additional text or explanation.

#     CONTEXT: {document}
#     QUESTION: {question}
#     <|eot_id|><|start_header_id|>assistant<|end_header_id|>
#     """,
#     input_variables=["question", "document"],
# )

# # Instantiate the retrieval grader
# retrieval_grader = prompt | llm | JsonOutputParser()

# # Define the question and retrieve the document
# question = "What is agent memory"
# docs = retriever.invoke(question)
# doc_txt = docs[1].page_content

# # Invoke the LLM with the prompt
# print("LLM output:", retrieval_grader.invoke({"question": question, "document": doc_txt}))


In [16]:
### Retrieval Grader

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "agent memory"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'yes'}


In [42]:
### Generate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

llm = ChatOllama(model=local_llm, temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
# question = "What is NIST?"
question = "How many times ram world appears in the book?"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

According to the provided context, the name "Ram" appears 2533 times in the holy Scripture Guru Granth Sahib.
